In [125]:
import numpy as np
import os

In [131]:
def read_xyz(xyz):
    coordlist=[]
    with open(xyz,'r') as inp:
        i=0
        for line in inp:   
            if i>=2:
                splitline=line.split()
                coordlist.append(splitline[1:4])
            i+=1
    return np.array(coordlist).astype(float)

def write_xyz(coordlist,name):
    with open(name,'w') as f:
        f.write(str(len(coordlist))+'\n#\n')
        for cd in coordlist:
            f.write('C '+str(cd[0])+' '+str(cd[1])+' '+str(cd[2])+'\n')

In [137]:
## read xyz files ##
path='tests/'
file2=path+'relaxed_new.xyz'
filename='master.xyz'

##loop over all master.xyz
#file1='master.xyz'
for subdir, dirs, files in sorted(os.walk(path)):
    for file in files:
        if file==filename:
            f=os.path.join(subdir, file)
            print('reading '+f)
            
            file1=f
            cl1=read_xyz(file1)
            cl2=read_xyz(file2)  

            ## get some atoms with same id (manually) ##
            same_ids=np.array([[1,8,267,31,45,124],[506,504,859,465,541,533]])
            same_ids-=1


            ## Rotate struture if necessairy ##
            xrotate=False
            if xrotate:
                ys=cl2[:,1]*np.cos(3.1415)-cl2[:,2]*np.sin(3.1415)
                zs=cl2[:,1]*np.sin(3.1415)+cl2[:,2]*np.cos(3.1415)

                cl2[:,1]=ys
                cl2[:,2]=zs

            ## calculate translation vector from correlated atoms ##    
            vec=np.zeros(3)
            for i in range(len(same_ids[0])):
                vec+=(cl1[same_ids[0,i]]-cl2[same_ids[1,i]])
            vec/=i+1

            ## apply translation ##
            cl2+=vec



            ## correlate all atoms ##
            ids=[]
            for coords in cl1:
                diff=np.sqrt(np.sum((coords-cl2)**2,axis=1))
                d=diff[np.argmin(diff)]
                ids.append(np.argmin(diff))


            ## calculate and apply translation of all correlated atoms ##
            ## this finetunes the translation ##
            vec=np.zeros(3)
            for i in range(len(cl1)):
                vec+=(cl1[i]-cl2[ids[i]])
            vec/=i+1
            cl2+=vec

            ## calculate difference ##

            totdiff=0
            for i in range(len(cl1)):
                totdiff+=np.sqrt(np.sum((cl1[i]-cl2[ids[i]])**2))
            print(totdiff/len(cl1))

            ## write new xyz ##
            write_xyz(cl2,path+'relaxed_shifted.xyz')

reading tests/master.xyz
0.2657367095576301
reading tests/001/master.xyz
0.2608953363638614
reading tests/002/master.xyz
0.25670141908857635
reading tests/003/master.xyz
0.2532796053533918
reading tests/005/master.xyz
0.24749602096283313
reading tests/010/master.xyz
0.23552732746372682
reading tests/015/master.xyz
0.22704525252546706
reading tests/020/master.xyz
0.22065882912885457
reading tests/030/master.xyz
0.21087527767816935
reading tests/040/master.xyz
0.2034800945149704
reading tests/050/master.xyz
0.194339781871286
reading tests/070/master.xyz
0.2657367095576301
reading tests/080/master.xyz
0.17674223328181193
reading tests/090/master.xyz
0.2657367095576301
reading tests/099/master.xyz
0.2657367095576301
reading tests/100/master.xyz
0.22917682067902237


array([-0.15836366,  0.01133166,  0.29158731])